In [1]:
from sklearn.datasets import fetch_openml

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

c:\Users\siddh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
# from ucimlrepo import fetch_ucirepo 

# # fetch dataset 
# optical_recognition_of_handwritten_digits = fetch_ucirepo(id=80) 
  
# # data (as pandas dataframes) 
# X = optical_recognition_of_handwritten_digits.data.features 
# y = optical_recognition_of_handwritten_digits.data.targets 
  
# # metadata 
# print(optical_recognition_of_handwritten_digits.metadata) 

# # variable information 
# print(optical_recognition_of_handwritten_digits.variables) 

In [4]:
# y= y.to_numpy()
# y =y.ravel()

In [5]:
import numpy as np
import random

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# rf_classifier.fit(X_train, y_train)

# y_pred = rf_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [8]:
# svm_classifier = SVC(kernel='linear', C=1.0)

# svm_classifier.fit(X_train, y_train)

# y_pred = svm_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [9]:
# xgb_classifier = xgb.XGBClassifier()
# xgb_classifier.fit(X_train, y_train)

# y_pred = xgb_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [10]:
def cal_pop_fitness(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:,columns]
        rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_classifier.fit(X_train_ind, y_train)

        y_pred = rf_classifier.predict(X_test.iloc[:,columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def fit_svm(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:, columns]
        svm_classifier = SVC(kernel='linear', C=1.0)
        svm_classifier.fit(X_train_ind, y_train)

        y_pred = svm_classifier.predict(X_test.iloc[:, columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def fit_xgboost(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:, columns]
        xgb_classifier = xgb.XGBClassifier()
        xgb_classifier.fit(X_train_ind, y_train)

        y_pred = xgb_classifier.predict(X_test.iloc[:, columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -1
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)

    for k in range(offspring_size[0]):
        # Always perform one-point crossover.
        crossover_point = np.random.randint(1, parents.shape[1])
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = np.random.choice([-1, 0, 1], 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

In [11]:
col_len = 112

sol_per_pop = 100
num_parents_mating = 50

pop_size = (sol_per_pop,col_len)

max_value = 783 
std_dev = 150
mean = max_value / 2

new_population = np.random.normal(mean, std_dev, pop_size).astype(int)

new_population[new_population > max_value] = max_value
new_population[new_population < 0] = 0

print(new_population)

[[415 268 472 ... 349 294 439]
 [469 531 461 ... 400 455 589]
 [454 423 373 ... 313 472 144]
 ...
 [652 353 417 ... 106 525 311]
 [479 237 478 ... 336 640 314]
 [400 320 409 ... 420  96 207]]


In [12]:
y=y.astype('int64')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train[:10000]
X_test = X_test[:3000]
y_train = y_train[:10000]
y_test = y_test[:3000]

In [14]:
num_generations = 20
for generation in range(num_generations):
    # Measing the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population, X_train,y_train,X_test,y_test)

    print(fitness)

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                      num_parents_mating)
    
    # print(parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], col_len))


    # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)

    # print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation  # should be offspring_mutation

    # The best result in the current iteration.
    if generation%10 ==0:
        print("Generation : ", generation)
        # print(offspring_crossover)
        print()

[0.9083333333333333, 0.9143333333333333, 0.886, 0.909, 0.9033333333333333, 0.8963333333333333, 0.9036666666666666, 0.91, 0.9193333333333333, 0.9153333333333333, 0.9066666666666666, 0.921, 0.8973333333333333, 0.899, 0.9116666666666666, 0.907, 0.9126666666666666, 0.9233333333333333, 0.9136666666666666, 0.9136666666666666, 0.9056666666666666, 0.9086666666666666, 0.9133333333333333, 0.9086666666666666, 0.9006666666666666, 0.905, 0.9116666666666666, 0.8976666666666666, 0.919, 0.9103333333333333, 0.911, 0.919, 0.9176666666666666, 0.9086666666666666, 0.912, 0.9043333333333333, 0.9036666666666666, 0.8933333333333333, 0.9136666666666666, 0.9016666666666666, 0.9056666666666666, 0.9113333333333333, 0.9073333333333333, 0.8993333333333333, 0.919, 0.9106666666666666, 0.9053333333333333, 0.9103333333333333, 0.9146666666666666, 0.9013333333333333, 0.9086666666666666, 0.918, 0.9023333333333333, 0.889, 0.902, 0.8993333333333333, 0.8786666666666667, 0.9153333333333333, 0.9053333333333333, 0.913, 0.888666

KeyboardInterrupt: 

In [15]:
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population, X_train, y_train, X_test,y_test)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.max(fitness))

# print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx[0][0]])

Best solution fitness :  0.933


In [16]:
print("Best solution : ", new_population[best_match_idx, :])

Best solution :  [[[457 603 536 553  55 432 653 378 304 635 698 275 265 260 539 210 260
   519 238 308 187 267 305 368 525 434 454 689 217 492 539 349 185 345
   577 129 443 417 665 339 628 265 313 281 375 319 313 565 283 577 516
   430 593 334 525 287 354 460 409 470 622 417 425 309 307 600 276 475
   216 269 414 210 479 290 474 300 576 374  14 272 336 337 463 256 526
   236 359 348 467 342 324 217 350 333 567 277 553 352 421 438 233 231
   370 487 521 270 485 551 355 409 460 221]]]


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9667619047619047


In [25]:
feat = new_population[best_match_idx, :]

In [26]:
feat[0][0]

array([457, 603, 536, 553,  55, 432, 653, 378, 304, 635, 698, 275, 265,
       260, 539, 210, 260, 519, 238, 308, 187, 267, 305, 368, 525, 434,
       454, 689, 217, 492, 539, 349, 185, 345, 577, 129, 443, 417, 665,
       339, 628, 265, 313, 281, 375, 319, 313, 565, 283, 577, 516, 430,
       593, 334, 525, 287, 354, 460, 409, 470, 622, 417, 425, 309, 307,
       600, 276, 475, 216, 269, 414, 210, 479, 290, 474, 300, 576, 374,
        14, 272, 336, 337, 463, 256, 526, 236, 359, 348, 467, 342, 324,
       217, 350, 333, 567, 277, 553, 352, 421, 438, 233, 231, 370, 487,
       521, 270, 485, 551, 355, 409, 460, 221])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train.iloc[:,feat[0][0]], y_train)

y_pred = rf_classifier.predict(X_test.iloc[:,feat[0][0]])

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9505714285714286
